<h1>Background Removal Model</h1>
<p>
    The model uses the 'rembg' library which implements and trains a U2-NET neural network model to process images.
</p>
<p>
    The functions here work as follows - 
    <ul>
        <li>download_image(img_url): This function takes in the url of the image and then downloads the image and returns it.</li>
        <li>save_image(img, path): This function takes in the return value from the download_image function and saves it to a path mentioned in the code.</li>
        <li>remove_background(img, alpha_matting=True, alpha_matting_foreground_threshold=50): This function takes in the image as input and then removes the background and saves it to a path.</li>
        <li>change_background(foreground_img, background_img_url): This function takes in the background_removed_image and downloads the new background image and makes it a new background.</li>
    </ul>
</p>
<p>
    NOTE: Please make sure to run this code in local machine. Keep two directories 'original' and 'masked' and this code file in the same directory and then run it.
</p>

In [1]:
import os
from io import BytesIO
from PIL import Image
import requests
from rembg import remove

C:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
def download_image(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    return img

In [3]:
def save_image(img, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    if img.mode == 'RGBA':
        img = img.convert('RGB')
        path = os.path.splitext(path)[0] + '.png'
    img.save(path)

In [4]:
def remove_background(img, alpha_matting=True, alpha_matting_foreground_threshold=50):
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    subject = remove(img_byte_arr, 
                     alpha_matting=alpha_matting, 
                     alpha_matting_foreground_threshold=alpha_matting_foreground_threshold)
    return Image.open(BytesIO(subject))

In [5]:
def change_background(foreground_img, background_img_url):
    background_img = download_image(background_img_url)
    background_img = background_img.resize(foreground_img.size)
    background_img.paste(foreground_img, (0, 0), foreground_img)
    return background_img

In [6]:
img_url = 'https://nationaltoday.com/wp-content/uploads/2020/12/National-Horse-Day-1-1200x834.jpg'
background_img_url = 'https://img.freepik.com/free-photo/digital-lavender-natural-landscape_23-2150538352.jpg'
img_name = img_url.split('/')[-1]

original_img = download_image(img_url)
save_image(original_img, 'original/' + img_name)

foreground_img = remove_background(original_img)
save_image(foreground_img, 'masked/' + img_name)

final_img = change_background(foreground_img, background_img_url)
save_image(final_img, 'new_background/background.jpg')